In [1]:
import pyglet
pyglet.options['shadow_window'] = False
import os
import numpy as np
import trimesh
from tf import transformations

from pyrender import IntrinsicsCamera,\
                     DirectionalLight, SpotLight, PointLight,\
                     MetallicRoughnessMaterial,\
                     Primitive, Mesh, Node, Scene,\
                     Viewer, OffscreenRenderer

from grasp_proposal_from_mesh import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def pose_matrix(euler, translation):
    pose_matrix = transformations.euler_matrix(euler[0], euler[1], euler[2])
    pose_matrix[:3,3] = translation
    return np.asarray(pose_matrix)

def rotate_vector(euler, vector):
    return np.matmul(transformations.euler_matrix(euler[0], euler[1], euler[2])[:3,:3],vector)

In [3]:
file_name = "/home/dof6/.gazebo/models/t_shape/meshes/t_shape.dae"

object_trimesh, _ = load_dae_to_trimesh(file_name)
gripper_contact_list, gripper_quaternion_list, gripper_width_list, quality_list, extra_information_list = sample_antipodal_grasp(object_trimesh,number_of_contact=100,number_of_quaternion=8)
print(np.sum(quality_list))
print(len(quality_list))
print(np.sum(quality_list)/len(quality_list))

149.0
2400
0.06208333333333333


In [4]:
def generate_image(object_trimesh):
    object_face_colors = np.asarray([[1.,0.,0.]])*np.ones(object_trimesh.faces.shape)
    object_trimesh.visual.face_colors = object_face_colors
    object_mesh = Mesh.from_trimesh(object_trimesh, smooth=False)
    object_pose = pose_matrix(np.pi*(2.*np.random.uniform(size=[3,])-1.),[0.,0.,0.1])
    
    point_l = PointLight(color=np.ones(3), intensity=10.0)

    cam_width = 640
    cam_height = 480
    cam_fx = 521.179233
    cam_fy = 493.033034
    cam_cx = cam_width/2.
    cam_cy = cam_height/2.
    cam = IntrinsicsCamera(cam_fx,cam_fy,cam_cx,cam_cy)

    cam_euler = np.pi/12.*(2.*np.random.uniform(size=[3,])-1.)
    cam_euler[2] = 0.
    cam_pose = pose_matrix(cam_euler,rotate_vector(cam_euler,[0.,0.,1.0]))

    wood_trimesh = trimesh.load('./background_models/wood.obj')
    wood_trimesh.vertices = wood_trimesh.vertices*10.
    wood_mesh = Mesh.from_trimesh(wood_trimesh)

    scene = Scene(ambient_light=np.array([0.02, 0.02, 0.02, 10.0]))

    object_node = scene.add(object_mesh, pose=object_pose)
    point_l_node = scene.add(point_l, pose=cam_pose)
    wood_node = scene.add(wood_mesh)
    cam_node = scene.add(cam, pose=cam_pose)

    r = OffscreenRenderer(viewport_width=cam_width, viewport_height=cam_height,)
    color, depth = r.render(scene)
    r.delete()
    return color, depth, cam_pose, cam, object_pose
# color, depth, cam_pose, cam, object_pose = generate_image(object_trimesh)
# cam_fx,cam_fy,cam_cx,cam_cy = cam.fx, cam.fy, cam.cx, cam.cy

In [5]:
# plt.figure()
# plt.imshow(color)
# plt.figure()
# plt.imshow(depth)
# plt.colorbar()
# plt.show()

In [6]:
number_of_object_pose = 50
visualize = False
cropping_size = 25

for i in range(number_of_object_pose):
    color, depth, cam_pose, cam, object_pose = generate_image(object_trimesh)
    cam_fx,cam_fy,cam_cx,cam_cy = cam.fx, cam.fy, cam.cx, cam.cy
    cam_inv = np.linalg.inv(cam_pose)

    grasp_quality_list = []
    grasp_width_list = []
    grasp_depth_list = []
    eef_position_list = []
    eef_position_p_list = []
    eef_euler_list = []
    eef_quaternion_list = []
    all_eef_points_list = []
    all_eef_points_p_list = []
    cropped_color_image_list = []
    cropped_depth_image_list = []

    for gripper_contact, gripper_quaternion, gripper_width, quality, extra_information in zip(gripper_contact_list, gripper_quaternion_list, gripper_width_list, quality_list, extra_information_list):
        eef_position = object_pose[:3,3] + np.matmul(object_pose[:3,:3], gripper_contact)
        eef_quaternion = tf.transformations.quaternion_multiply(transformations.quaternion_from_matrix(object_pose), gripper_quaternion)
        all_eef_points = object_pose[:3,[3]] + np.matmul(object_pose[:3,:3], extra_information['all_gripper_points'].T)

        eef_position_cam_frame = cam_inv[:3,3] + np.matmul(cam_inv[:3,:3],eef_position)
        eef_quaternion_cam_frame = tf.transformations.quaternion_multiply(transformations.quaternion_from_matrix(cam_inv), eef_quaternion)
        gripper_depth = -eef_position_cam_frame[2]
        eef_euler_cam_frame = tf.transformations.euler_from_quaternion(eef_quaternion_cam_frame)
        approaching_direction = rotate_vector(eef_quaternion_cam_frame,[0.,0.,1.])

        if approaching_direction[2] > np.cos(np.pi/6.):
            quality = 0.

        eef_px = -cam_fx*eef_position_cam_frame[0]/eef_position_cam_frame[2] + cam_cx
        eef_py = cam_fy*eef_position_cam_frame[1]/eef_position_cam_frame[2] + cam_cy

        center_x = int(eef_px)
        center_y = int(eef_py)

        cropped_color_image = color[center_y-cropping_size:center_y+cropping_size,center_x-cropping_size:center_x+cropping_size,:]
        cropped_depth_image = depth[center_y-cropping_size:center_y+cropping_size,center_x-cropping_size:center_x+cropping_size,np.newaxis] - gripper_depth

        all_eef_points_cam_frame = cam_inv[:3,[3]] +  np.matmul(cam_inv[:3,:3],all_eef_points)
        all_eef_points_px = -cam_fx*all_eef_points_cam_frame[0,:]/all_eef_points_cam_frame[2,:] + cam_cx
        all_eef_points_py = cam_fy*all_eef_points_cam_frame[1,:]/all_eef_points_cam_frame[2,:] + cam_cy
        all_eef_points_p = np.concatenate([[all_eef_points_px],[all_eef_points_py]],axis=0).T
        eef_p = np.concatenate([[eef_px],[eef_py]],axis=0).T

        grasp_quality_list.append(quality)
        eef_position_list.append(eef_position_cam_frame)
        eef_position_p_list.append(eef_p)
        eef_quaternion_list.append(eef_quaternion_cam_frame)
        all_eef_points_list.append(all_eef_points_cam_frame)
        all_eef_points_p_list.append(all_eef_points_p)
        eef_euler_list.append(eef_euler_cam_frame)
        cropped_color_image_list.append(cropped_color_image.astype(np.float32)/255.)
        cropped_depth_image_list.append(cropped_depth_image)
        grasp_width_list.append(gripper_width)
        grasp_depth_list.append(gripper_depth)
        
        if quality==1.0:
            for _ in range(4):
                grasp_quality_list.append(quality)
                eef_position_list.append(eef_position_cam_frame)
                eef_position_p_list.append(eef_p)
                eef_euler_list.append(eef_euler_cam_frame)
                eef_quaternion_list.append(eef_quaternion_cam_frame)
                all_eef_points_list.append(all_eef_points_cam_frame)
                all_eef_points_p_list.append(all_eef_points_p)
                cropped_color_image_list.append(cropped_color_image.astype(np.float32)/255.)
                cropped_depth_image_list.append(cropped_depth_image)
                grasp_width_list.append(gripper_width)
                grasp_depth_list.append(gripper_depth)

        if quality==1.0:
            # TYPE 1:
            random_direction = 2*np.pi*np.random.uniform()
            center_x = int(eef_px + cropping_size*np.cos(random_direction))
            center_y = int(eef_py + cropping_size*np.sin(random_direction))
            cropped_color_image = color[center_y-cropping_size:center_y+cropping_size,center_x-cropping_size:center_x+cropping_size,:]
            cropped_depth_image = depth[center_y-cropping_size:center_y+cropping_size,center_x-cropping_size:center_x+cropping_size,np.newaxis] - gripper_depth + 0.05*(2.*np.random.uniform() - 1.)

            grasp_quality_list.append(0.0)
            eef_position_list.append(eef_position_cam_frame)
            eef_position_p_list.append(eef_p)
            eef_euler_list.append(eef_euler_cam_frame)
            eef_quaternion_list.append(eef_quaternion_cam_frame)
            all_eef_points_list.append(all_eef_points_cam_frame)
            all_eef_points_p_list.append(all_eef_points_p)
            cropped_color_image_list.append(cropped_color_image.astype(np.float32)/255.)
            cropped_depth_image_list.append(cropped_depth_image)
            grasp_width_list.append(gripper_width)
            grasp_depth_list.append(gripper_depth)
            
        if visualize:
            plt.figure()
            plt.imshow(cropped_color_image)
            plt.figure()
            plt.imshow(cropped_depth_image[:,:,0])
            plt.plot(cropping_size,cropping_size,'ko')
            plt.plot(all_eef_points_px - center_x + cropping_size,all_eef_points_py - center_y + cropping_size,'k-')
            plt.show()
            break
            
    if visualize:
        break
        
    print(len(grasp_quality_list))
    print(np.sum(grasp_quality_list)/len(grasp_quality_list))
    np.save("./grasp_data/t_shape/"+str(i)+"-"+str(number_of_object_pose)+".npy",{
        'grasp_quality_list' : grasp_quality_list,
        'eef_position_list' : eef_position_list,
        'eef_position_p_list' : eef_position_p_list,
        'eef_quaternion_list' : eef_quaternion_list,
        'eef_euler_list' : eef_euler_list,
        'all_eef_points_list' : all_eef_points_list,
        'all_eef_points_p_list' : all_eef_points_p_list,
        'cropped_color_image_list' : cropped_color_image_list,
        'cropped_depth_image_list' : cropped_depth_image_list,
        'grasp_width_list' : grasp_width_list,
        'grasp_depth_list' : grasp_depth_list,
        'cam_fx' : cam_fx,
        'cam_fy' : cam_fy,
        'cam_cx' : cam_cx,
        'cam_cy' : cam_cy,
        'cam_pose' : cam_pose
    })

2825
0.1504424778761062
2920
0.1780821917808219
2880
0.16666666666666666
2910
0.17525773195876287
2845
0.15641476274165203
2880
0.16666666666666666
3090
0.22330097087378642
2795
0.1413237924865832
2840
0.15492957746478872
2880
0.16666666666666666
2980
0.19463087248322147
2925
0.1794871794871795
2895
0.17098445595854922
2925
0.1794871794871795
2900
0.1724137931034483
3030
0.2079207920792079
2990
0.19732441471571907
2905
0.1738382099827883
3040
0.21052631578947367
2880
0.16666666666666666
2890
0.1695501730103806
2850
0.15789473684210525
2930
0.18088737201365188
2815
0.14742451154529307
3020
0.2052980132450331
2885
0.1681109185441941
3000
0.2
3045
0.21182266009852216
2830
0.1519434628975265
3075
0.21951219512195122
2925
0.1794871794871795
3075
0.21951219512195122
3090
0.22330097087378642
2915
0.17667238421955403
2885
0.1681109185441941
2920
0.1780821917808219
3035
0.20922570016474465
2895
0.17098445595854922
2965
0.1905564924114671
2885
0.1681109185441941
2940
0.1836734693877551
3025
0.20